# Wikidata Statement Nanopublication Generator

Generates nanopublications expressing structured statements using Wikidata entities and properties.

## Wikidata Statements
- **Subject**: A Wikidata entity or external URI
- **Property**: A Wikidata property (e.g., P31 = instance of)
- **Object**: A Wikidata entity or literal value

In [1]:
CONFIG_FILE = "../config/vbae208/vbae208_wikidata.json"
CONFIG_FILE = "../config/clenet2025/clenet2025_wikidata.json"
OUTPUT_DIR = "../output/wikidata"

In [2]:
import json
from pathlib import Path
from datetime import datetime, timezone
from rdflib import Dataset, Namespace, URIRef, Literal
from rdflib.namespace import RDF, RDFS, XSD, FOAF

NP = Namespace("http://www.nanopub.org/nschema#")
DCT = Namespace("http://purl.org/dc/terms/")
NT = Namespace("https://w3id.org/np/o/ntemplate/")
NPX = Namespace("http://purl.org/nanopub/x/")
PROV = Namespace("http://www.w3.org/ns/prov#")
ORCID = Namespace("https://orcid.org/")
WD = Namespace("http://www.wikidata.org/entity/")
WDT = Namespace("http://www.wikidata.org/prop/direct/")

WIKIDATA_TEMPLATE = URIRef("https://w3id.org/np/RA-46AJ9o1Nxal80CEsWhOjXwr2G3Ox5wPT6z3KxGbS7o")
PROV_TEMPLATE = URIRef("https://w3id.org/np/RA7lSq6MuK_TIC6JMSHvLtee3lpLoZDOqLJCLXevnrPoU")
PUBINFO_TEMPLATE_1 = URIRef("https://w3id.org/np/RA0J4vUn_dekg-U1kK3AOEt02p9mT2WO03uGxLDec1jLw")
PUBINFO_TEMPLATE_2 = URIRef("https://w3id.org/np/RAukAcWHRDlkqxk7H2XNSegc1WnHI569INvNr-xdptDGI")

print("✓ Setup complete")

✓ Setup complete


In [3]:
with open(CONFIG_FILE, 'r', encoding='utf-8') as f:
    config = json.load(f)

metadata = config.get('metadata', {})
print(f"✓ Loaded {len(config['nanopublications'])} Wikidata nanopubs")

✓ Loaded 5 Wikidata nanopubs


In [4]:
def format_entity(entity):
    """Format a Wikidata entity or external URI."""
    if 'uri' in entity:
        return URIRef(entity['uri'])
    elif 'id' in entity:
        return WD[entity['id']]
    else:
        return Literal(str(entity))

def format_property(prop):
    """Format a Wikidata property."""
    if 'uri' in prop:
        return URIRef(prop['uri'])
    elif 'id' in prop:
        return WDT[prop['id']]
    else:
        raise ValueError(f"Property must have 'uri' or 'id': {prop}")

def create_wikidata_nanopub(np_config, metadata):
    TEMP_NP = Namespace("http://purl.org/nanopub/temp/np/")
    
    this_np = URIRef("http://purl.org/nanopub/temp/np/")
    head_graph = URIRef("http://purl.org/nanopub/temp/np/Head")
    assertion_graph = URIRef("http://purl.org/nanopub/temp/np/assertion")
    provenance_graph = URIRef("http://purl.org/nanopub/temp/np/provenance")
    pubinfo_graph = URIRef("http://purl.org/nanopub/temp/np/pubinfo")
    
    author_uri = ORCID[metadata['creator_orcid']]
    
    ds = Dataset()
    ds.bind("this", "http://purl.org/nanopub/temp/np/")
    ds.bind("sub", TEMP_NP)
    ds.bind("np", NP)
    ds.bind("dct", DCT)
    ds.bind("nt", NT)
    ds.bind("npx", NPX)
    ds.bind("xsd", XSD)
    ds.bind("rdfs", RDFS)
    ds.bind("orcid", ORCID)
    ds.bind("prov", PROV)
    ds.bind("foaf", FOAF)
    ds.bind("wd", WD)
    ds.bind("wdt", WDT)
    
    # HEAD
    head = ds.graph(head_graph)
    head.add((this_np, RDF.type, NP.Nanopublication))
    head.add((this_np, NP.hasAssertion, assertion_graph))
    head.add((this_np, NP.hasProvenance, provenance_graph))
    head.add((this_np, NP.hasPublicationInfo, pubinfo_graph))
    
    # ASSERTION
    assertion = ds.graph(assertion_graph)
    
    for stmt in np_config.get('statements', []):
        subject = format_entity(stmt['subject'])
        predicate = format_property(stmt['property'])
        obj = format_entity(stmt['object'])
        
        # Main statement
        assertion.add((subject, predicate, obj))
        
        # Labels
        if stmt['subject'].get('label'):
            assertion.add((subject, RDFS.label, Literal(stmt['subject']['label'])))
        if stmt['object'].get('label') and stmt['object'].get('id'):
            assertion.add((obj, RDFS.label, Literal(stmt['object']['label'])))
    
    # PROVENANCE
    provenance = ds.graph(provenance_graph)
    provenance.add((assertion_graph, PROV.wasAttributedTo, author_uri))
    
    # PUBINFO
    pubinfo = ds.graph(pubinfo_graph)
    pubinfo.add((author_uri, FOAF.name, Literal(metadata['creator_name'])))
    
    now = datetime.now(timezone.utc).strftime("%Y-%m-%dT%H:%M:%S.000Z")
    pubinfo.add((this_np, DCT.created, Literal(now, datatype=XSD.dateTime)))
    pubinfo.add((this_np, DCT.creator, author_uri))
    pubinfo.add((this_np, DCT.license, URIRef("https://creativecommons.org/licenses/by/4.0/")))
    pubinfo.add((this_np, NPX.wasCreatedAt, URIRef("https://sciencelive4all.org/")))
    
    # isPartOf in pubinfo for Wikidata
    is_part_of = metadata.get('is_part_of', {})
    if is_part_of.get('uri'):
        pubinfo.add((this_np, DCT.isPartOf, URIRef(is_part_of['uri'])))
        if is_part_of.get('label'):
            pubinfo.add((URIRef(is_part_of['uri']), NT.hasLabelFromApi, Literal(is_part_of['label'])))
    
    label = np_config.get('nanopub_label', 'Wikidata Statement')
    pubinfo.add((this_np, RDFS.label, Literal(label)))
    
    pubinfo.add((this_np, NT.wasCreatedFromTemplate, WIKIDATA_TEMPLATE))
    pubinfo.add((this_np, NT.wasCreatedFromProvenanceTemplate, PROV_TEMPLATE))
    pubinfo.add((this_np, NT.wasCreatedFromPubinfoTemplate, PUBINFO_TEMPLATE_1))
    pubinfo.add((this_np, NT.wasCreatedFromPubinfoTemplate, PUBINFO_TEMPLATE_2))
    
    return ds, label

print("✓ Function defined")

✓ Function defined


In [5]:
Path(OUTPUT_DIR).mkdir(parents=True, exist_ok=True)
generated_files = []

for np_config in config['nanopublications']:
    ds, label = create_wikidata_nanopub(np_config, metadata)
    output_file = Path(OUTPUT_DIR) / f"{np_config['id']}.trig"
    ds.serialize(destination=str(output_file), format='trig')
    generated_files.append(output_file)
    print(f"✓ Generated: {output_file}")

print(f"\nTotal: {len(generated_files)} nanopublications")

✓ Generated: ../output/wikidata/wikidata_clenet_subject_qc.trig
✓ Generated: ../output/wikidata/wikidata_clenet_subject_ecology.trig
✓ Generated: ../output/wikidata/wikidata_clenet_subject_ecomodel.trig
✓ Generated: ../output/wikidata/wikidata_clenet_subject_networks.trig
✓ Generated: ../output/wikidata/wikidata_qiskit_instance.trig

Total: 5 nanopublications


In [6]:
if generated_files:
    print(f"Preview of {generated_files[0]}:\n")
    print("=" * 80)
    with open(generated_files[0], 'r') as f:
        print(f.read())

Preview of ../output/wikidata/wikidata_clenet_subject_qc.trig:

@prefix dct: <http://purl.org/dc/terms/> .
@prefix foaf: <http://xmlns.com/foaf/0.1/> .
@prefix np: <http://www.nanopub.org/nschema#> .
@prefix npx: <http://purl.org/nanopub/x/> .
@prefix nt: <https://w3id.org/np/o/ntemplate/> .
@prefix orcid: <https://orcid.org/> .
@prefix prov: <http://www.w3.org/ns/prov#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix sub: <http://purl.org/nanopub/temp/np/> .
@prefix wd: <http://www.wikidata.org/entity/> .
@prefix wdt: <http://www.wikidata.org/prop/direct/> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

sub:assertion {
    <https://arxiv.org/abs/2504.03866> rdfs:label "Addressing Ecological Challenges from a Quantum Computing Perspective" ;
        wdt:P921 wd:Q17995793 .

    wd:Q17995793 rdfs:label "quantum computing" .
}

sub:Head {
    sub: a np:Nanopublication ;
        np:hasAssertion sub:assertion ;
        np:hasProvenance sub:provenance ;
        np:ha

In [7]:
PUBLISH = True  # Set to True when ready to publish
USE_TEST_SERVER = False  # Set to False for production
if PUBLISH:
    from nanopub import Nanopub, NanopubConf, load_profile
    
    profile = load_profile()
    print(f"Loaded profile: {profile.name}")
    
    conf = NanopubConf(profile=profile, use_test_server=USE_TEST_SERVER)
    
    for trig_file in generated_files:
        np_obj = Nanopub(rdf=trig_file, conf=conf)
        np_obj.sign()
        
        signed_path = trig_file.with_suffix('.signed.trig')
        np_obj.store(signed_path)
        print(f"✓ Signed: {signed_path}")
        
        np_obj.publish()
        print(f"✓ Published: {np_obj.source_uri}")
else:
    print("Publishing disabled. Set PUBLISH = True to enable.")
    print("\nManual signing:")
    for f in generated_files:
        print(f"  nanopub sign {f}")
        print(f"  nanopub publish {f.stem}.signed.trig")

Loaded profile: Anne Fouilloux
✓ Signed: ../output/wikidata/wikidata_clenet_subject_qc.signed.trig
✓ Published: https://w3id.org/np/RATUK6U22XZup3c59qcDG01Y5-thlsDiNB2V_EF2cjP_k
✓ Signed: ../output/wikidata/wikidata_clenet_subject_ecology.signed.trig
✓ Published: https://w3id.org/np/RARyn2yoYNNKneSM65a9iVJP16cQZQp0nh4JQnIH0Mgw0
✓ Signed: ../output/wikidata/wikidata_clenet_subject_ecomodel.signed.trig
✓ Published: https://w3id.org/np/RAhMaTeAPRJrZ01QcJCAuzQrAP3VPthS9j0Y9CtUAVv4Y
✓ Signed: ../output/wikidata/wikidata_clenet_subject_networks.signed.trig
✓ Published: https://w3id.org/np/RAAxOl1poIZL9ofGT8pVJdgLVdLDppEtBGCOu_AFaGvW4
✓ Signed: ../output/wikidata/wikidata_qiskit_instance.signed.trig
✓ Published: https://w3id.org/np/RA8MDxdFRSI4dlES3nWrnbTgDemmxlWWOhGDy3G5XdbJA
